# Iris ML E2E

In [1]:
import os, uuid, datetime
import tensorflow as tf
from pyspark.sql import functions as F
import json
import numpy as np

from hops import hdfs, featurestore, model, experiment, tensorboard, constants, util, serving
from hops.model import Metric

print('Tensorflow version', tf.__version__)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
57,application_1582107121686_0002,pyspark,idle,Link,Link


SparkSession available as 'spark'.
Tensorflow version 1.14.0

In [2]:
# Names

FS_NAME = 'ml_monitoring_featurestore'

MODELS_DIR = "Models/Iris/"
IRIS_RAW_DATA_DIR = "Raw_Datasets/Iris/"

IRIS_TRAIN_DATASET_NAME = "iris_train_dataset"
IRIS_FG_NAME = "iris_train_all_features"
IRIS_FG_DESCRIPTION = "Iris training dataset with all features"
IRIS_MODEL_NAME="Iris"
IRIS_TOPIC_NAME = "iris_ml_topic"

## Data preparation

In [3]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

COL_NAMES = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
SPECIES = ['setosa', 'versicolor', 'virginica']
LABEL_NAME = 'species'

### Training data

In [4]:
# download training data to local
train_filename = TRAIN_URL.split('/')[-1]
train_local_path = os.getcwd() + '/' + train_filename
train_path = tf.keras.utils.get_file(train_local_path, TRAIN_URL)

8192/2194 [================================================================================================================] - 0s 0us/step

In [ ]:
# copy data to hdfs
hdfs.copy_to_hdfs(train_local_path, IRIS_RAW_DATA_DIR, overwrite=True)

In [ ]:
# create feature group with all the features for training

# read and cast
train_df = spark.read.csv(hdfs.project_path() + IRIS_RAW_DATA_DIR + train_filename, header=True).toDF(*COL_NAMES)
train_df = train_df.select(*(F.col(c).cast("float").alias(c) for c in COL_NAMES))

print(train_df.show(4))
print(train_df.printSchema())

# create feature group
featurestore.create_featuregroup(train_df, IRIS_FG_NAME,
                                 description=IRIS_FG_DESCRIPTION,
                                 descriptive_statistics=True,
                                 feature_correlation=True,
                                 feature_histograms=True,
                                 cluster_analysis=True)

In [ ]:
# inspect feature group
train_fg = featurestore.get_featuregroup(IRIS_FG_NAME)
print(train_fg.show(4))
print(train_fg.printSchema())

In [ ]:
# create feature training dataset
featurestore.create_training_dataset(train_fg, IRIS_TRAIN_DATASET_NAME,
                                     data_format="tfrecords",
                                     descriptive_statistics=True,
                                     feature_correlation=True,
                                     feature_histograms=True,
                                     cluster_analysis=True)

### Test data

In [ ]:
# download test data to local
test_filename = TEST_URL.split('/')[-1]
test_local_path = os.getcwd() + '/' + test_filename
test_path = tf.keras.utils.get_file(test_local_path, TEST_URL)

In [ ]:
# copy data to hdfs
hdfs.copy_to_hdfs(test_local_path, IRIS_RAW_DATA_DIR, overwrite=True)

In [ ]:
# read and cast
test_hdfs_path = hdfs.project_path() + IRIS_RAW_DATA_DIR + test_filename
test_df = spark.read.csv(test_hdfs_path, header=True).toDF(*COL_NAMES)
test_df = test_df.select(*(F.col(c).cast("float").alias(c) for c in COL_NAMES))

print(test_df.show(4))
print(test_df.printSchema())

In [ ]:
# copy preprocessed test data to hdfs
def get_prep_path(path, suffix):
    point_idx = path.rfind('.')
    return path[:point_idx] + suffix + path[point_idx:]

point_idx = test_filename.rfind('.')
test_prep_dir = test_filename[:point_idx] + "_prep"
test_prep_hdfs_path = test_hdfs_path[:test_hdfs_path.rfind('/') + 1] + test_prep_dir
test_df.write.option("header", "true").mode("overwrite").csv(test_prep_hdfs_path)

## Experiments

In [10]:
# config

EVALUATION_METRIC="accuracy"
DATASET_SHUFFLE_BUFF_SIZE = 10000

In [ ]:
def load_data(batch_size):
    
    def stack_features_vector_from_dict(row):
        label = tf.cast(row.pop(LABEL_NAME), tf.int64) # pop label
        labels = tf.one_hot(label, len(SPECIES)) # one hot encoding
        features = tf.stack(list(row.values()), axis=1)
        return features, labels

    def stack_features_vector(features, labels):
        features = tf.stack(list(features.values()), axis=1)
        labels = tf.one_hot(tf.cast(labels, tf.int64), len(SPECIES))  # one hot encoding
        return features, labels
    
    def decode(pb):
        return tf.parse_single_example(pb, tf_record_schema)

    # training data
    print('Loading training data...')
    dataset_dir = featurestore.get_training_dataset_path(IRIS_TRAIN_DATASET_NAME)
    input_files = tf.gfile.Glob(dataset_dir + "/part-r-*") # the tf records are written in a distributed manner using partitions
    tf_record_schema = featurestore.get_training_dataset_tf_record_schema(IRIS_TRAIN_DATASET_NAME) # tf record schemas are managed by the feature store
    train_dataset = tf.data.TFRecordDataset(input_files).map(decode).shuffle(DATASET_SHUFFLE_BUFF_SIZE).batch(batch_size).repeat(1).map(stack_features_vector_from_dict)
    
    # test data
    print('Loading test data...')
    hdfs.copy_to_local(test_prep_hdfs_path)
    test_dataset = tf.data.experimental.make_csv_dataset(test_prep_dir + "/part-*", batch_size=batch_size, num_epochs=1, label_name=LABEL_NAME).map(stack_features_vector)
    
    # n_features, n_classes
    features, _ = next(iter(train_dataset))
    n_features, n_classes = features.shape[1].value, len(SPECIES)
    
    return train_dataset, test_dataset, n_features, n_classes

In [ ]:
def create_model(n_features, n_classes):
    
    # nn parameters
    n_hidden_1 = 256 # 1st layer number of neurons
    n_hidden_2 = 128 # 1st layer number of neurons

    # weights
    weights = {
      'h1': tf.Variable(tf.random_normal([n_features, n_hidden_1]), name='h1'),
      'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='h2'),
      'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name='wout')
    }
    
    # biases
    biases = {
      'b1': tf.Variable(tf.random_normal([n_hidden_1]), name='b1'),
      'b2': tf.Variable(tf.random_normal([n_hidden_2]), name='b2'),
      'out': tf.Variable(tf.random_normal([n_classes]), name='bout')
    }
    
    # forward propagation
    def forward_propagation(x):
        
        # hidden layer 1
        layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
        layer_1 = tf.nn.relu(layer_1)

        # hidden layer 2
        layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
        layer_2 = tf.nn.relu(layer_2)
        
        # output: fully connected layer
        out_layer = tf.matmul(layer_2, weights['out']) + biases['out'] 
        
        return out_layer
    
    return forward_propagation

In [ ]:
def accuracy(sess, yprobs, X, features, labels, train_conf=None):
    
    def _accuracy(preds, labels):
        matches = tf.equal(tf.argmax(preds, axis=1), tf.argmax(labels, axis=1))
        return tf.reduce_mean(tf.cast(matches, "float"))
    
    batch_accuracies = 0
    batch_count = 0
    while True:
        try:
            batch_x, batch_y = sess.run([features, labels])
            if train_conf is not None:
                sess.run(train_conf['train_op'], feed_dict={X: batch_x, train_conf['y']: batch_y}) # train step
            preds = sess.run(yprobs, feed_dict={X: batch_x})
#             acc, acc_op = tf.metrics.accuracy(labels=tf.argmax(batch_y, axis=1), predictions=tf.argmax(preds, axis=1))
            batch_accuracies += _accuracy(preds, batch_y) # batch acc
            batch_count += 1
        except tf.errors.OutOfRangeError:
            break

    if batch_accuracies == 0 or batch_count == 0:
        return -1
    else:
        return batch_accuracies / batch_count

def train_model(sess, model, train_dataset, test_dataset, n_features, n_classes, hyperparams):
    
    print('Training model...')

    # tensorboard
#     summ_writer = tf.summary.FileWriter(tensorboard.logdir() + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"), sess.graph)
#     with tf.name_scope('performance'):
#         test_accuracy_ph = tf.placeholder(tf.float32, shape=None, name='test_accuracy_summary')
#         train_accuracy_ph = tf.placeholder(tf.float32, shape=None, name='train_accuracy_summary')
#         train_accuracy_summary = tf.summary.scalar('train_accuracy', train_accuracy_ph)
#         test_accuracy_summary = tf.summary.scalar('test_accuracy', test_accuracy_ph)
#         performance_summaries = tf.summary.merge([train_accuracy_summary, test_accuracy_summary])

    # inputs
    X = tf.placeholder("float", shape=[None, n_features], name='X') # features
    y = tf.placeholder("float", shape=[None, n_classes], name='y') # labels

    # ouputs
    yhat = model(X) # logits (,n_classes)
    yprobs = tf.nn.softmax(yhat) # predictions -> probabilities (,n_classes)
    ypreds = tf.argmax(yprobs, axis=1)
    yprobs_ordered, idxs = tf.nn.top_k(yprobs, n_classes)
    table = tf.contrib.lookup.index_to_string_table_from_tensor(tf.constant([str(i) for i in range(n_classes)]))
    ypredict_ordered = table.lookup(tf.to_int64(idxs)) # predictions -> all indexes ordered by probs (,n_classes)
    
    # backward propagation
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
    optimizer = tf.train.GradientDescentOptimizer(hyperparams['learning_rate'])
    train_op = optimizer.minimize(cost)
    
    # create iterator and initializers
    iter = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
    features, labels = iter.get_next()
    train_init_op = iter.make_initializer(train_dataset)
    test_init_op = iter.make_initializer(test_dataset)
    
    for epoch in range(hyperparams['epochs']):
        
        # training
        sess.run(train_init_op) # initialize with training data
        epoch_train_accuracy = accuracy(sess, yprobs, X, features, labels, train_conf={'train_op': train_op, 'y': y }) # get accuracy while training
        
        # epoch testing
        sess.run(test_init_op) # switch to test data
        epoch_test_accuracy = accuracy(sess, yprobs, X, features, labels) # without training
        
        # tensorboard
#         summ = sess.run(performance_summaries, feed_dict={train_accuracy_summary: epoch_train_accuracy, test_accuracy_summary: epoch_test_accuracy})
#         summ_writer.add_summary(summ, epoch)
        
        print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%" % (epoch + 1, 100. * epoch_train_accuracy.eval(), 100. * epoch_test_accuracy.eval()))

    # test accuracy
    sess.run(test_init_op) # switch to test data
    test_accuracy = accuracy(sess, yprobs, X, features, labels)
    
    return { EVALUATION_METRIC: test_accuracy.eval() }, { 'X': X, 'ypredict_ordered': ypredict_ordered, 'yprobs_ordered': yprobs_ordered, 'ypreds': ypreds }

In [ ]:
def export_model(sess, metrics, signature):
    
    export_path = '{}/{}model-{}'.format(os.getcwd(), MODELS_DIR, str(uuid.uuid4()))
    print('Exporting trained model to: {}'.format(export_path))
    
    builder = tf.saved_model.builder.SavedModelBuilder(export_path)

    # build signature_def_map
    clas_inputs = { tf.saved_model.signature_constants.CLASSIFY_INPUTS: tf.saved_model.utils.build_tensor_info(signature['X']) }
    clas_outputs = { tf.saved_model.signature_constants.CLASSIFY_OUTPUT_CLASSES: tf.saved_model.utils.build_tensor_info(signature['ypredict_ordered']),
                     tf.saved_model.signature_constants.CLASSIFY_OUTPUT_SCORES: tf.saved_model.utils.build_tensor_info(signature['yprobs_ordered']) }

    classification_signature = (tf.saved_model.signature_def_utils.build_signature_def(
                                    inputs=clas_inputs, outputs=clas_outputs,
                                    method_name=tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME))

    pred_inputs = { tf.saved_model.signature_constants.PREDICT_INPUTS: tf.saved_model.utils.build_tensor_info(signature['X'])}
    pred_outputs = { tf.saved_model.signature_constants.PREDICT_OUTPUTS: tf.saved_model.utils.build_tensor_info(signature['ypreds'])}

    prediction_signature = (tf.saved_model.signature_def_utils.build_signature_def(
                              inputs=pred_inputs, outputs=pred_outputs,
                              method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

    legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')
    builder.add_meta_graph_and_variables(
          sess, [tf.saved_model.tag_constants.SERVING],
          signature_def_map={
              'predict_instances': prediction_signature,
              tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: classification_signature,
          },
          legacy_init_op=legacy_init_op)

    builder.save()

    model.export(export_path, IRIS_MODEL_NAME, metrics=metrics)

In [ ]:
def iris_ml_experiment(learning_rate, epochs, batch_size):
    
    sess = tf.InteractiveSession()
    hyperparams = { 'learning_rate': learning_rate, 'epochs': epochs, 'batch_size': batch_size }
    
    # load data
    train_dataset, test_dataset, n_features, n_classes = load_data(hyperparams['batch_size'])
    print('Loading data: DONE')

    # create model
    iris_model = create_model(n_features, n_classes)

    # train model
    sess.run(tf.global_variables_initializer())
    metrics, signature = train_model(sess, iris_model, train_dataset, test_dataset, n_features, n_classes, hyperparams)
    print('Training model: DONE')

    # export model
    export_model(sess, metrics, signature)
    print('Exporting model: DONE')
    
    sess.close()
    
    return metrics

def iris_ml_single_experiment():
    learning_rate = 0.01
    batch_size = 32
    epochs = 1
    
    return iris_ml_experiment(learning_rate, epochs, batch_size)

In [ ]:
# search dictionary
hyperparams={'learning_rate': [0.01, 0.001, 0.02], 'batch_size': [32, 64, 128], 'epochs': [50, 75, 100] }

# single experiment
# experiment.launch(iris_ml_single_experiment, name='iris ml single experiment', metric_key=EVALUATION_METRIC)

# grid search experiment
experiment.grid_search(iris_ml_experiment, hyperparams, direction='max', name='iris ml experiment', optimization_key=EVALUATION_METRIC)

# differential evolution
# experiment.differential_evolution(iris_ml_experiment, hyperparams, direction=experiment.Direction.MAX, optimization_key='accuracy')
# experiment.differential_evolution(
#     iris_ml_experiment, 
#     hyperparams, 
#     direction='max',
#     generations=4,
#     population=5,                      
#     name='iris ml experiment', 
#     optimization_key='accuracy',
#     description='Demonstration of iris ml hyperparameters optimization',
#     local_logdir=False)

# Serve the best model

### Create Kafka topic

In [4]:
TOPIC_SCHEMA = { "name": "inferencelog", "type": "record", "fields": [
    {"name": "modelId", "type": "int"},
    {"name": "modelName", "type": "string"},
    {"name": "modelVersion", "type": "int"},
    {"name": "requestTimestamp", "type": "long"},
    {"name": "responseHttpCode", "type": "int"},
    {"name": "inferenceResponse", "type": "string"},
    {"name": "servingType", "type": "string"}
] }

In [5]:
from hops.exceptions import RestAPIError
def create_kafka_topic(topic_name, topic_schema):
    headers = {constants.HTTP_CONFIG.HTTP_CONTENT_TYPE: constants.HTTP_CONFIG.HTTP_APPLICATION_JSON}
    method = constants.HTTP_CONFIG.HTTP_POST
    resource_url = constants.DELIMITERS.SLASH_DELIMITER + \
                   constants.REST_CONFIG.HOPSWORKS_REST_RESOURCE + constants.DELIMITERS.SLASH_DELIMITER + \
                   constants.REST_CONFIG.HOPSWORKS_PROJECT_RESOURCE + constants.DELIMITERS.SLASH_DELIMITER + \
                   hdfs.project_id() + constants.DELIMITERS.SLASH_DELIMITER + \
                   constants.REST_CONFIG.HOPSWORKS_KAFKA_RESOURCE + constants.DELIMITERS.SLASH_DELIMITER + \
                   constants.REST_CONFIG.HOPSWORKS_SUBJECTS_RESOURCE + constants.DELIMITERS.SLASH_DELIMITER + \
                   topic_name + constants.DELIMITERS.SLASH_DELIMITER + \
                   "versions"

    schema = { "schema" : topic_schema }
    response = util.send_request(method, resource_url, data=json.dumps(schema), headers=headers)
    response_object = response.json()
    if response.status_code >= 400:
        error_code, error_msg, user_msg = util._parse_rest_error(response_object)
        raise RestAPIError("Could not perform action on job's execution (url: {}), server response: \n "
                           "HTTP code: {}, HTTP reason: {}, error code: {}, error msg: {}, user msg: {}".format(
            resource_url, response.status_code, response.reason, error_code, error_msg, user_msg))
    return response_object

> **NOTE:** Replace with custom schema.

> - Kafka topic creation via API is not currently working due to limited permission // HTTP code: 403, HTTP reason: Forbidden, error code: 200014, error msg: Token not issued for this recipient

> - Add it manually via Hopsworks UI -> Copy paste the following schema:

In [6]:
def convert_fs_schema_to_avro_schema(fs_schema, field_name='fs_schema'):
    fields = []
    for feature in fs_schema:
        fields.append({"name": feature['name'], "type": feature['type'].lower()})
    return { "name": field_name, "type": { "name": field_name, "type": "record", "fields": fields } }

def convert_tf_record_schema_to_spark_schema(tf_schema, spark_int_type=constants.SPARK_CONFIG.SPARK_INTEGER_TYPE,
                                             spark_float_type=constants.SPARK_CONFIG.SPARK_DOUBLE_TYPE,
                                             spark_string_type=constants.SPARK_CONFIG.SPARK_STRING_TYPE):
    fields = []
    for key in tf_schema.keys():    
        tftype = tf_schema[key]
    
        # int - array(int)
        if tftype.dtype ==  tf.FixedLenFeature([], tf.int64).dtype:
            if tftype.shape == []:
                fields.append({"metadata": {}, "name": key, "nullable": True, "type": spark_int_type})
            else:
                fields.append({"metadata": {}, "name": key, "nullable": True, "type": {"containsNull": True, "elementType": spark_int_type, "type": "array"}})
            continue
        if tftype.dtype == tf.VarLenFeature(tf.int64).dtype:
            fields.append({"metadata": {}, "name": key, "nullable": True, "type": {"containsNull": True, "elementType": spark_int_type, "type": "array"}})
            continue
            
        # float - array(float)
        if tftype.dtype ==  tf.FixedLenFeature([], tf.float32).dtype:
            if tftype.shape == []:
                fields.append({"metadata": {}, "name": key, "nullable": True, "type": spark_float_type})
            else:
                fields.append({"metadata": {}, "name": key, "nullable": True, "type": {"containsNull": True, "elementType": spark_float_type, "type": "array"}})
            continue
        if tftype.dtype == tf.VarLenFeature(tf.float32).dtype:
            fields.append({"metadata": {}, "name": key, "nullable": True, "type": {"containsNull": True, "elementType": spark_float_type, "type": "array"}})
            continue
        
        # string - array(string)
        # NOTE: VarLenFeature is used for both string and array(string) so real type cannot be inferred. We assume it is an array. Check "_get_dataframe_tf_record_schema_json" method.
        if tftype.dtype == tf.VarLenFeature(tf.string).dtype:
            fields.append({"metadata": {}, "name": key, "nullable": True, "type": {"containsNull": True, "elementType": spark_string_type, "type": "array"}})
            continue
    return { "fields": fields, "type": "struct" }

def convert_spark_schema_to_avro_schema(s_schema, field_name='spark_schema'):
    fields = []
    for field in s_schema['fields']:
        ftype = field['type']
        fschema = { "name": field['name'] }
        
        if isinstance(ftype, str):
            fschema['type'] = ftype
        elif isinstance(ftype, dict):
            if ftype['type'] == 'array':
                fschema['type'] = ftype['type']
                fschema['items'] = ftype['elementType']
        fields.append(fschema)
    return { "name": field_name, "type": { "name": field_name, "type": "record", "fields": fields } }

In [7]:
# get inference schema from featurestore metadata
ds_version = 1
fs_meta = featurestore.get_featurestore_metadata(featurestore=FS_NAME)
fs_schema = [vars(f) for f in fs_meta.training_datasets[IRIS_TRAIN_DATASET_NAME + "_" + str(ds_version)].features]
inference_request_schema = convert_fs_schema_to_avro_schema(fs_schema)
print("Avro schema from featurestore metadata:\n", inference_request_schema, "\n")

# get inference schema from TFRecord schema
tfrecord_schema = featurestore.get_training_dataset_tf_record_schema(IRIS_TRAIN_DATASET_NAME, featurestore=IRIS_FG_NAME)
spark_schema = convert_tf_record_schema_to_spark_schema(tfrecord_schema)
inference_request_schema_2 = convert_spark_schema_to_avro_schema(spark_schema, field_name='inferenceRequest')
print("Avro schema from training dataset schema:\n", inference_request_schema_2,"\n")

# add to base schema
TOPIC_SCHEMA["fields"].append(inference_request_schema_2)
TOPIC_SCHEMA = json.dumps(TOPIC_SCHEMA)

Avro schema from featurestore metadata:
 {'name': 'fs_schema', 'type': {'name': 'fs_schema', 'type': 'record', 'fields': [{'name': 'sepal_length', 'type': 'float'}, {'name': 'sepal_width', 'type': 'float'}, {'name': 'petal_length', 'type': 'float'}, {'name': 'petal_width', 'type': 'float'}, {'name': 'species', 'type': 'float'}]}} 

Avro schema from training dataset schema:
 {'name': 'inferenceRequest', 'type': {'name': 'inferenceRequest', 'type': 'record', 'fields': [{'name': 'sepal_length', 'type': 'double'}, {'name': 'sepal_width', 'type': 'double'}, {'name': 'petal_length', 'type': 'double'}, {'name': 'petal_width', 'type': 'double'}, {'name': 'species', 'type': 'double'}]}}

> **IMPORTANT:** Create kafka topic not working API.

> - Create kafka topic manually with the following parameters

In [8]:
# create kafka topic # NOT WORKING API
# response = create_kafka_topic(IRIS_TOPIC_NAME, TOPIC_SCHEMA)
# print(response)

# COPY THROUGH HOPSWORKS UI
print("Topic name: ", IRIS_TOPIC_NAME)
print("Schema:\n", TOPIC_SCHEMA)

Topic name:  iris_ml_topic
Schema:
 {"name": "inferencelog", "type": "record", "fields": [{"name": "modelId", "type": "int"}, {"name": "modelName", "type": "string"}, {"name": "modelVersion", "type": "int"}, {"name": "requestTimestamp", "type": "long"}, {"name": "responseHttpCode", "type": "int"}, {"name": "inferenceResponse", "type": "string"}, {"name": "servingType", "type": "string"}, {"name": "inferenceRequest", "type": {"name": "inferenceRequest", "type": "record", "fields": [{"name": "sepal_length", "type": "double"}, {"name": "sepal_width", "type": "double"}, {"name": "petal_length", "type": "double"}, {"name": "petal_width", "type": "double"}, {"name": "species", "type": "double"}]}}]}

### Create or update model

In [9]:
# get best model
best_model = model.get_best_model(IRIS_MODEL_NAME, EVALUATION_METRIC, Metric.MAX)

print('Model name: ' + best_model['name'])
print('Model version: ' + str(best_model['version']))
print(best_model['metrics'])

name 'EVALUATION_METRIC' is not defined
Traceback (most recent call last):
NameError: name 'EVALUATION_METRIC' is not defined



> **IMPORTANT:** HTTP code: 400, HTTP reason: Bad Request, error code: 240010, error msg: Topic provided cannot be used for Serving logging, user msg: inferenceschema required

In [14]:
# serve the model
response = serving.create_or_update(MODELS_DIR, IRIS_MODEL_NAME, topic_name=IRIS_TOPIC_NAME, serving_type="TENSORFLOW", model_version=best_model['version'])

Creating a serving for model Iris ...
Serving for model Iris successfully created

In [15]:
# get serving status
serving.get_status(IRIS_MODEL_NAME)

'Stopped'

# Start model serving

In [25]:
if serving.get_status(IRIS_MODEL_NAME) == 'Stopped':
    serving.start(IRIS_MODEL_NAME)

import time
time.sleep(10) # Let the serving startup correctly

Starting serving with name: Iris...
Serving with name: Iris successfully started

# Predictions on-demand

In [4]:
def generate_instance():
    sl = round(np.random.uniform(1,5), 1)
    sw = round(np.random.uniform(1,4), 1)
    pl = round(np.random.uniform(1,5), 1)
    pw = round(np.random.uniform(1,3), 1)
    return [sl, sw, pl, pw]

In [5]:
def batch_predictions(instances, signature_name):
    instances = [generate_instance() for i in range(instances)]
    data = { "signature_name": signature_name,
             "instances": instances }
    response = serving.make_inference_request(IRIS_MODEL_NAME, data)
    for inst, pred in zip(instances, response['predictions']):
        print("Instance: ", inst)
        print(pred)

print("# Classification with scores")
batch_predictions(3, tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY)
    
print("\n# Direct classification")
batch_predictions(3, 'predict_instances')

# Classification with scores
Instance:  [2.8, 2.7, 1.4, 1.1]
{'scores': [1.0, 0.0, 0.0], 'classes': ['0', '1', '2']}
Instance:  [4.3, 2.1, 4.3, 2.3]
{'scores': [0.999824, 0.000176003829, 1.79565079e-17], 'classes': ['2', '1', '0']}
Instance:  [1.9, 1.9, 1.5, 1.6]
{'scores': [1.0, 1.39763413e-24, 1.33207961e-26], 'classes': ['0', '2', '1']}

# Direct classification
Instance:  [3.6, 1.1, 1.6, 2.7]
0
Instance:  [1.9, 1.3, 1.7, 1.9]
0
Instance:  [3.0, 3.1, 3.8, 2.0]
1

# Consuming predictions through Kafka

In [6]:
import json
from hops import kafka, tls
from confluent_kafka import Consumer

NUM_FEATURES = len(COL_NAMES)-1

config = kafka.get_kafka_default_config()
config['default.topic.config'] = {'auto.offset.reset': 'earliest'}
topics = [IRIS_TOPIC_NAME]

In [7]:
consumer = Consumer(config)
consumer.subscribe(topics, on_assign = lambda _, partitions: print('Assignment:', partitions))

In [8]:
json_schema = kafka.get_schema(IRIS_TOPIC_NAME)
avro_schema = kafka.convert_json_schema_to_avro(json_schema)

In [9]:
for i in range(0, 10):
    msg = consumer.poll(timeout=1.0)
    if msg is not None:
        value = msg.value()
        try:
            event_dict = kafka.parse_avro_msg(value, avro_schema)
            prediction = json.loads(event_dict["inferenceResponse"])["predictions"][0]
            print("serving: {}, version: {}, timestamp: {},"\
                  "\nrequest: {},\nprediction: {}, http_response_code: {},"\
                  " serving_type: {}\n".format(
                       event_dict["modelName"],
                       event_dict["modelVersion"],
                       event_dict["requestTimestamp"],
                       event_dict["inferenceRequest"],
                       prediction,
                       event_dict["responseHttpCode"],
                       event_dict["servingType"]))
        except Exception as e:
            print("A message was read but there was an error parsing it")
            print(e)
    else:
        print("... timeout, no more messages to read from topic")

... timeout, no more messages to read from topic
... timeout, no more messages to read from topic
... timeout, no more messages to read from topic
... timeout, no more messages to read from topic
... timeout, no more messages to read from topic
... timeout, no more messages to read from topic
... timeout, no more messages to read from topic
Assignment: [TopicPartition{topic=iris_ml_topic,partition=0,offset=-1001,error=None}]
serving: Iris, version: 58, timestamp: 1581515022883,
request: {"signature_name": "serving_default", "instances": [[2.8, 2.7, 1.4, 1.1], [4.3, 2.1, 4.3, 2.3], [1.9, 1.9, 1.5, 1.6]]},
prediction: {'scores': [1.0, 0.0, 0.0], 'classes': ['0', '1', '2']}, http_response_code: 200, serving_type: TENSORFLOW

serving: Iris, version: 58, timestamp: 1581515023045,
request: {"signature_name": "predict_instances", "instances": [[3.6, 1.1, 1.6, 2.7], [1.9, 1.3, 1.7, 1.9], [3.0, 3.1, 3.8, 2.0]]},
prediction: 0, http_response_code: 200, serving_type: TENSORFLOW

... timeout, no m

In [24]:
# from io import BytesIO, StringIO
# from avro.io import DatumReader, BinaryDecoder
# import avro.schema
    
# print("\nVALUE:")
# print(value)

# print("\nCustom schema:")
# custom_avro_schema = kafka.convert_json_schema_to_avro(avro_type_struct)
# print(custom_avro_schema)

# base_avro_schema = kafka.convert_json_schema_to_avro(base_avro_type_struct)
# print(base_avro_schema)

# print("\nRequest:")
# reader = DatumReader(base_avro_schema)
# message_bytes = BytesIO(value)
# decoder = BinaryDecoder(message_bytes)
# request = reader.read(decoder)
# print(request['inferenceRequest'])
# print(type(json.loads(request['inferenceRequest'])['instances'][0][0]))